# Estrategias Avanzadas de Gestión de Memoria

## 🌌 Escenario: USS Enterprise - Sistemas Avanzados de Memoria

El Teniente Comandante Data ha desarrollado **nuevos sistemas de gestión de memoria** para optimizar la bitácora estelar. En esta misión exploraremos técnicas que van más allá del simple truncamiento:

- 🔄 **Reducción Automática**: El sistema decide cuándo reducir el historial
- 📝 **Resumen Inteligente**: Condensar información antigua sin perderla
- 💾 **Persistencia de Estado**: Guardar y restaurar conversaciones completas
- 🧠 **Memory de Pizarra**: Capturar decisiones críticas automáticamente
- 🌀 **Memoria a Largo Plazo**: Recordar información entre sesiones
- ⚙️ **Context Providers**: Inyectar contexto automáticamente en cada invocación

🖖 *"La eficiencia no consiste en hacer más con menos, sino en hacer lo correcto con precisión." - Spock*

---

## 📦 Configuración Inicial

In [3]:
import asyncio
import os
import json
import pandas as pd
from datetime import datetime
from abc import ABC, abstractmethod
from dataclasses import dataclass, asdict
from typing import Optional
from agent_framework.openai import OpenAIChatClient

print("✅ Librerías cargadas correctamente")

✅ Librerías cargadas correctamente


In [4]:
# Configuración del cliente Azure OpenAI
base_url = os.getenv("AZURE_OPENAI_ENDPOINT")
api_key = os.getenv("AZURE_OPENAI_API_KEY")
model_id = os.getenv("AZURE_OPENAI_DEPLOYMENT")

# Crear cliente
client = OpenAIChatClient(
    base_url=base_url,
    api_key=api_key,
    model_id=model_id
)

# Crear agente de bitácora avanzado
agent = client.create_agent(
    name="SistemaAvanzadoBitacora",
    instructions="""Eres el sistema avanzado de bitácora de la USS Enterprise. Procesas información de forma eficiente, 
    respondes de manera concisa y estás siempre listo para aprender nuevos datos sobre la tripulación."""
)

print("✅ Sistema Avanzado de Bitácora activado")
print("🚀 USS Enterprise lista para la misión")

✅ Sistema Avanzado de Bitácora activado
🚀 USS Enterprise lista para la misión


---

# 📝 Estrategia 1: Resumen Inteligente con IA

## 📖 Concepto

En lugar de **eliminar**, **resume** información antigua usando IA. Conserva lo importante mientras reduce tokens.

**Cómo funciona:**
1. Cuando historial supera umbral (ej: 15 mensajes)
2. Los mensajes antiguos se envían al modelo
3. El modelo genera resumen conciso
4. Se mantienen mensajes recientes completos

**Caso Star Trek:** Data resume informe técnico largo del Teniente La Forge manteniendo los puntos críticos.

In [5]:
async def resumir_mensajes(mensajes, contexto=""):
    """Usa el agente para resumir una lista de mensajes"""
    texto_completo = "\n".join([f"{m}" for m in mensajes])
    
    prompt = f"""Resume de forma concisa estas entradas de bitácora estelar.
Mantén solo información crítica: eventos importantes, decisiones, anomalías.
Máximo 3-4 líneas.

ENTRADAS:
{texto_completo}

RESUMEN CONCISO:"""
    
    thread_resumen = agent.get_new_thread()
    resultado = await agent.run(prompt, thread=thread_resumen)
    return resultado.text

print("✅ Función resumir_mensajes definida")

✅ Función resumir_mensajes definida


In [6]:
print("\n" + "="*70)
print("🎬 ESCENA: Data Resume Informe de Ingeniería - Ventana Deslizante")
print("Situación: Geordi La Forge envía reporte técnico línea por línea")
print("Sistema: Mantiene ventana de 4 items (comprimiendo antiguos automáticamente)")
print("="*70)

# Lista de mensajes que iremos añadiendo de 3 en 3
mensajes = [
    "Sistema de propulsión warp operativo",
    "Núcleo de materia-antimateria al 98% de eficiencia",
    "Detectada pequeña grieta en conducto de plasma Sector 7G",
    "Convertidor EPS en cubierta 12 reporta sobrecalentamiento",
    "Blindaje de antimateria recalibrado exitosamente",
    "Sistema de enfriamiento primario necesitará servicio importante",
    "Todo el personal de ingeniería reporta buena moral",
    "Nave lista para operaciones de máxima potencia",
    "Sistemas de comunicación operando normalmente",
]

# Estado: lista de items en memoria (pueden ser mensajes o resúmenes)
memory_items = []  # Cada item es {"tipo": "mensaje"|"resumen", "contenido": str}
resumen_actual = None  # Almacena el resumen de lo antiguo cuando existe
contador_resumenes = 0

VENTANA_MAX = 4  # Máximo de items que caben en memoria activa

print(f"\n📌 Configuración: Ventana máxima = {VENTANA_MAX} items")
print("   • Los últimos 3 mensajes siempre sin comprimir")
print("   • Todo lo anterior se comprime en 1 resumen\n")

# Procesar mensajes de 3 en 3
for lote_num in range(1, len(mensajes) // 3 + 2):
    inicio = (lote_num - 1) * 3
    fin = min(inicio + 3, len(mensajes))
    lote = mensajes[inicio:fin]
    
    if not lote:
        break
    
    print("\n" + "="*70)
    print(f"📥 LOTE #{lote_num}: Agregando {len(lote)} mensajes")
    print("="*70)
    
    # Añadir los 3 mensajes del lote
    for msg in lote:
        memory_items.append({"tipo": "mensaje", "contenido": msg})
        print(f"   ✓ Agregado: {msg[:50]}...")
    
    # ALGORITMO DE VENTANA DESLIZANTE
    # Si excedimos la ventana, comprimir los más antiguos
    while len(memory_items) > VENTANA_MAX:
        print(f"\n   ⚠️ Memoria llena ({len(memory_items)} items). Necesito comprimir...")
        
        # Identificar cuánto necesito comprimir
        items_a_comprimir = len(memory_items) - VENTANA_MAX + 1
        
        # Extraer los items a comprimir (más antiguos)
        items_comprimibles = memory_items[:items_a_comprimir]
        mensajes_para_resumir = [item["contenido"] for item in items_comprimibles]
        
        print(f"   📦 Resumiendo {len(mensajes_para_resumir)} items antiguos...")
        for idx, msg in enumerate(mensajes_para_resumir, 1):
            print(f"      [{idx}] {msg[:45]}...")
        
        # Generar resumen de lo antiguo
        resumen_actual = await resumir_mensajes(mensajes_para_resumir)
        contador_resumenes += 1
        
        print(f"\n   📝 RESUMEN GENERADO (v{contador_resumenes}):")
        print(f"      {resumen_actual[:100]}...")
        
        # Reemplazar los items antiguos por 1 resumen
        memory_items = [{"tipo": "resumen", "contenido": resumen_actual}] + memory_items[items_a_comprimir:]
        
        print(f"   ✅ Memoria comprimida: {len(items_comprimibles)} items → 1 resumen")
    
    # Mostrar estado actual de memoria
    print(f"\n📊 ESTADO ACTUAL DE MEMORIA ({len(memory_items)} items):")
    print("-"*70)
    for idx, item in enumerate(memory_items, 1):
        if item["tipo"] == "resumen":
            print(f"   [{idx}] 📦 RESUMEN: {item['contenido'][:60]}...")
        else:
            print(f"   [{idx}] 📝 MENSAJE: {item['contenido'][:60]}...")
    
    # CONSULTA AL AGENTE: después de cada lote de 3 mensajes
    print(f"\n🤖 CONSULTA AL AGENTE (Ve {len(memory_items)} items en memoria):")
    print("-"*70)
    
    # Construir el contexto para el agente
    contexto_agente = ""
    for item in memory_items:
        if item["tipo"] == "resumen":
            contexto_agente += f"[RESUMEN ANTERIOR]\n{item['contenido']}\n\n"
        else:
            contexto_agente += f"• {item['contenido']}\n"
    
    thread = agent.get_new_thread()
    pregunta = f"Basándote en SOLO esta información en tu memoria (que incluye resumen comprimido + últimos mensajes):\n\n{contexto_agente}\n¿Cuál es el STATUS actual del motor warp? Responde en una línea."
    
    resultado = await agent.run(pregunta, thread=thread)
    print(f"Agent Response: {resultado.text}")
    print("="*70)

print(f"\n✨ Demostración completada")
print(f"📊 Resúmenes generados: {contador_resumenes}")
print(f"✅ Sistema demostró: Compresión automática de ventana de {VENTANA_MAX} items")


🎬 ESCENA: Data Resume Informe de Ingeniería - Ventana Deslizante
Situación: Geordi La Forge envía reporte técnico línea por línea
Sistema: Mantiene ventana de 4 items (comprimiendo antiguos automáticamente)

📌 Configuración: Ventana máxima = 4 items
   • Los últimos 3 mensajes siempre sin comprimir
   • Todo lo anterior se comprime en 1 resumen


📥 LOTE #1: Agregando 3 mensajes
   ✓ Agregado: Sistema de propulsión warp operativo...
   ✓ Agregado: Núcleo de materia-antimateria al 98% de eficiencia...
   ✓ Agregado: Detectada pequeña grieta en conducto de plasma Sec...

📊 ESTADO ACTUAL DE MEMORIA (3 items):
----------------------------------------------------------------------
   [1] 📝 MENSAJE: Sistema de propulsión warp operativo...
   [2] 📝 MENSAJE: Núcleo de materia-antimateria al 98% de eficiencia...
   [3] 📝 MENSAJE: Detectada pequeña grieta en conducto de plasma Sector 7G...

🤖 CONSULTA AL AGENTE (Ve 3 items en memoria):
------------------------------------------------------------

---

# 📋 Estrategia 2: Memoria de Pizarra (Whiteboard)

## 📖 Concepto

Una **pizarra** captura información crítica que persiste incluso si el historial se trunca:
- ✅ **Requisitos**: Lo que se necesita hacer
- ✅ **Decisiones**: Elecciones importantes tomadas
- ✅ **Acciones**: Tareas pendientes o completadas
- ⚠️ **Alertas**: Información crítica

**Caso Star Trek:** Tablero de situación táctica del puente muestra solo información crítica en tiempo real.

In [7]:
class PizarraMision:
    """Captura información crítica que persiste siempre"""
    
    def __init__(self, max_items=10):
        self.max_items = max_items
        self.requisitos = []
        self.decisiones = []
        self.acciones_pendientes = []
        self.alertas = []
    
    def agregar_requisito(self, texto):
        self.requisitos.append(texto)
        self._limitar_items(self.requisitos)
    
    def agregar_decision(self, texto):
        self.decisiones.append(texto)
        self._limitar_items(self.decisiones)
    
    def agregar_accion(self, texto):
        self.acciones_pendientes.append(texto)
        self._limitar_items(self.acciones_pendientes)
    
    def agregar_alerta(self, texto):
        self.alertas.append(texto)
        self._limitar_items(self.alertas)
    
    def _limitar_items(self, lista):
        while len(lista) > self.max_items:
            lista.pop(0)
    
    def obtener_contexto(self):
        contexto = "\n=== PIZARRA DE MISIÓN ==="
        
        if self.requisitos:
            contexto += "\n\n📋 REQUISITOS:"
            for r in self.requisitos:
                contexto += f"\n  • {r}"
        
        if self.decisiones:
            contexto += "\n\n✓ DECISIONES:"
            for d in self.decisiones:
                contexto += f"\n  • {d}"
        
        if self.acciones_pendientes:
            contexto += "\n\n⏳ ACCIONES PENDIENTES:"
            for a in self.acciones_pendientes:
                contexto += f"\n  • {a}"
        
        if self.alertas:
            contexto += "\n\n⚠️ ALERTAS:"
            for alerta in self.alertas:
                contexto += f"\n  • {alerta}"
        
        contexto += "\n========================\n"
        return contexto
    
    def mostrar(self):
        print(self.obtener_contexto())

print("✅ Clase PizarraMision definida")

✅ Clase PizarraMision definida


In [8]:
print("\n" + "="*70)
print("🎬 ESCENA: Misión de Exploración - Sector 441")
print("Situación: Capitán Picard planifica misión crítica")
print("="*70)

pizarra = PizarraMision(max_items=5)
thread_mision = agent.get_new_thread()

print("\n📝 CAPITÁN PICARD DEFINE PARÁMETROS DE LA MISIÓN:\n")

# Capturar información
pizarra.agregar_requisito("Explorar anomalía subespacial en coordenadas 441-7")
print("✓ Requisito: Explorar anomalía subespacial")

pizarra.agregar_decision("Escudos al máximo durante aproximación")
print("✓ Decisión: Escudos al máximo")

pizarra.agregar_accion("Data debe analizar patrones de radiación")
print("✓ Acción: Data analizará radiación")

pizarra.agregar_alerta("Posible interferencia temporal detectada - CRÍTICO")
print("✓ Alerta: Interferencia temporal")

pizarra.agregar_decision("Geordi La Forge lidera equipo de ingeniería")
print("✓ Decisión: Geordi al mando de ingeniería")

# Mostrar pizarra
print("\n" + "="*70)
print("📊 TABLERO DE SITUACIÓN TÁCTICA ACTUAL")
print("="*70)
pizarra.mostrar()

# Usar pizarra con agente
print("🤖 CONSULTA DEL CAPITÁN:\n")
contexto_pizarra = pizarra.obtener_contexto()
pregunta = "Basándote en la pizarra de misión, ¿cuáles son las prioridades críticas?"
mensaje_completo = f"{contexto_pizarra}\n\nCAPITÁN PICARD: {pregunta}"

resultado_pizarra = await agent.run(mensaje_completo, thread=thread_mision)

print("📨 Pregunta:", pregunta)
print("\n💬 Respuesta del Sistema de Bitácora:")
print("="*70)
print(resultado_pizarra.text)
print("="*70)
print("\n✅ Sistema tiene acceso automático a información crítica")
print("✅ Información persiste incluso si el historial se truncara")


🎬 ESCENA: Misión de Exploración - Sector 441
Situación: Capitán Picard planifica misión crítica

📝 CAPITÁN PICARD DEFINE PARÁMETROS DE LA MISIÓN:

✓ Requisito: Explorar anomalía subespacial
✓ Decisión: Escudos al máximo
✓ Acción: Data analizará radiación
✓ Alerta: Interferencia temporal
✓ Decisión: Geordi al mando de ingeniería

📊 TABLERO DE SITUACIÓN TÁCTICA ACTUAL

=== PIZARRA DE MISIÓN ===

📋 REQUISITOS:
  • Explorar anomalía subespacial en coordenadas 441-7

✓ DECISIONES:
  • Escudos al máximo durante aproximación
  • Geordi La Forge lidera equipo de ingeniería

⏳ ACCIONES PENDIENTES:
  • Data debe analizar patrones de radiación

⚠️ ALERTAS:
  • Posible interferencia temporal detectada - CRÍTICO

🤖 CONSULTA DEL CAPITÁN:

📨 Pregunta: Basándote en la pizarra de misión, ¿cuáles son las prioridades críticas?

💬 Respuesta del Sistema de Bitácora:
Capitán, las prioridades críticas según la pizarra de misión son:

1. Mantener los escudos al máximo durante la aproximación a la anomalía su

## 📚 Concepto Arquitectónico: Pizarra vs Thread

### ¿Cuál es la diferencia real?

Hasta ahora hemos visto estrategias para **reducir el contexto dentro de un mismo thread**. Pero existe una distinción arquitectónica importante:

| Aspecto | **Thread** | **Pizarra** |
|--------|-----------|-----------|
| **Alcance** | Privado de 1 agente | Compartida entre N agentes |
| **Historial** | Acumula conversaciones privadas | Solo información actual |
| **Uso** | Memoria conversacional persistente | Información crítica compartida |
| **Visibilidad** | Solo ese agente la ve | Todos los agentes la ven |

### Escenarios de Uso

**Mono-Agente (1 solo agente):**
```
Thread = Todo lo que necesita
Pizarra = Información crítica adicional
```
El thread por sí solo suele ser suficiente. La pizarra es redundante.

**Multi-Agente (N agentes coordinados):**
```
Thread Agente 1  ← Conversación privada del Agente 1
Thread Agente 2  ← Conversación privada del Agente 2
       ↓
    Pizarra Compartida ← Lo que AMBOS deben saber (sin exponerse mutuamente)
```
Aquí es donde cobra sentido la Pizarra: compartir contexto crítico sin mezclar threads privados.

### Ejemplo Práctico: Misión Star Trek

En un equipo de exploración real:
- **Picard tiene su thread:** Conversación privada con Data sobre decisiones tácticas
- **Geordi tiene su thread:** Conversación privada sobre problemas del motor
- **Pizarra compartida:** "Escudos al máximo - Interferencia temporal detectada"

Ambos agentes ven la pizarra, pero NO ven el historial privado del otro.

### En Este Workshop

Las próximas estrategias usan **Pizarra en contexto mono-agente** para demostrar el concepto, pero el verdadero potencial se desbloquea en **sistemas multiagente coordinados**.

---

# 🧠 Estrategia 3: Memoria a Largo Plazo

## 📖 Concepto

Almacena información **entre sesiones**:
- 👤 **Perfil de Usuario**: Información estática (nombre, rango, preferencias)
- 📚 **Resumen de Chats**: Temas discutidos en sesiones previas

**Caso Star Trek:** Sistema reconoce al Capitán Picard y recuerda sus preferencias diplomáticas y arqueológicas.

In [12]:
class MemoriaLargoPlazoPerfil:
    """Almacena perfiles de tripulantes con memoria a largo plazo"""
    
    def __init__(self):
        self.perfiles = {
            "picard": {
                "nombre": "Capitán Jean-Luc Picard",
                "rango": "Capitán",
                "nave": "USS Enterprise NCC-1701-D",
                "preferencias": [
                    "Prefiere informes concisos y directos",
                    "Valora el protocolo diplomático",
                    "Interesado en arqueología, así que hazle referencias históricas"
                ],
                "sesiones_anteriores": [
                    "Sesión anterior: Discutió amenaza Borg en sector fronterizo",
                    "Sesión anterior: Revisó protocolos de primer contacto",
                    "Sesión anterior: Planificó misión diplomática Romulana"
                ]
            },
            "data": {
                "nombre": "Teniente Comandante Data",
                "rango": "Teniente Comandante",
                "nave": "USS Enterprise NCC-1701-D",
                "preferencias": [
                    "Proporciona análisis técnicos muy detallados",
                    "No entiende metáforas (humor no detectado)",
                    "Interesado en comprender humanidad y emociones"
                ],
                "sesiones_anteriores": [
                    "Sesión anterior: Analizó anomalías subespaciales",
                    "Sesión anterior: Optimizó algoritmos de sensores"
                ]
            }
        }
    
    def generar_contexto(self, user_id):
        """Genera contexto para inyectar al agente"""
        perfil = self.perfiles.get(user_id.lower(), None)
        if not perfil:
            return "Usuario desconocido"
        
        contexto = f"\n=== MEMORIA DE LARGO PLAZO ==="
        contexto += f"\n👤 USUARIO: {perfil['nombre']}"
        contexto += f"\n🎖️ RANGO: {perfil['rango']}"
        contexto += f"\n🚀 NAVE: {perfil['nave']}"
        contexto += "\n\n📋 PREFERENCIAS:"
        for pref in perfil['preferencias']:
            contexto += f"\n  • {pref}"
        contexto += "\n\n📚 SESIONES ANTERIORES:"
        for sesion in perfil['sesiones_anteriores'][-3:]:
            contexto += f"\n  • {sesion}"
        contexto += "\n==============================\n"
        return contexto

print("✅ Clase MemoriaLargoPlazoPerfil definida")

✅ Clase MemoriaLargoPlazoPerfil definida


In [13]:
print("\n" + "="*70)
print("🎬 ESCENA: Sistema Reconoce al Capitán Picard")
print("Situación: Inicio de sesión - Sistema carga perfil de memoria")
print("="*70)

memoria = MemoriaLargoPlazoPerfil()
user_id = "picard"
thread_memoria = agent.get_new_thread()

print(f"\n🔐 Identificando usuario: {user_id}")
print("📥 Cargando perfil de memoria a largo plazo...\n")

contexto_usuario = memoria.generar_contexto(user_id)
print(contexto_usuario)

# Primera consulta (Picard)
print("🎤 CAPITÁN PICARD: Necesito un análisis de situación actual")
pregunta1 = "¿Cuál es la situación actual? Sé conciso."
mensaje1 = f"{contexto_usuario}\n\nCAPITÁN: {pregunta1}"

resultado1 = await agent.run(mensaje1, thread=thread_memoria)
print(f"\n💬 Respuesta (personalizada a Picard):")
print(resultado1.text)

# Segunda consulta (Data) - otra sesión/thread
print("\n\n🎤 TENIENTE COMANDANTE DATA: Solicito estado de mis análisis previos")
user_id2 = "data"
thread_memoria_data = agent.get_new_thread()
contexto_usuario2 = memoria.generar_contexto(user_id2)

pregunta2 = "Según mis sesiones anteriores, ¿qué anomalías subespaciales he analizado y qué conclusión preliminar tenía? Sé conciso."
mensaje2 = f"{contexto_usuario2}\n\nDATA: {pregunta2}"

resultado2 = await agent.run(mensaje2, thread=thread_memoria_data)
print(f"\n💬 Respuesta (personalizada a Data):")
print(resultado2.text)

print("\n" + "="*70)
print("✅ SISTEMA PERSONALIZADO")
print("✅ Recordó preferencias del Capitán (informes concisos)")
print("✅ Recordó sesiones anteriores de Data (anomalías/sensores)")
print("✅ Contexto cargado automáticamente sin que el usuario lo mencionara")



🎬 ESCENA: Sistema Reconoce al Capitán Picard
Situación: Inicio de sesión - Sistema carga perfil de memoria

🔐 Identificando usuario: picard
📥 Cargando perfil de memoria a largo plazo...


=== MEMORIA DE LARGO PLAZO ===
👤 USUARIO: Capitán Jean-Luc Picard
🎖️ RANGO: Capitán
🚀 NAVE: USS Enterprise NCC-1701-D

📋 PREFERENCIAS:
  • Prefiere informes concisos y directos
  • Valora el protocolo diplomático
  • Interesado en arqueología, así que hazle referencias históricas

📚 SESIONES ANTERIORES:
  • Sesión anterior: Discutió amenaza Borg en sector fronterizo
  • Sesión anterior: Revisó protocolos de primer contacto
  • Sesión anterior: Planificó misión diplomática Romulana

🎤 CAPITÁN PICARD: Necesito un análisis de situación actual

💬 Respuesta (personalizada a Picard):
Capitán, situación actual:

- La USS Enterprise se mantiene en curso estándar, sin anomalías detectadas.
- Vigilancia aumentada en sector Borg, sin contactos recientes.
- Misión diplomática Romulana en preparación, equipo dipl

---

# ⚙️ Estrategia 4: Context Providers - Automatización Inteligente

## 📖 Concepto

Los **Context Providers** son componentes que se conectan al agente y:
1. **`invoking`**: Se ejecutan ANTES de invocar el agente
   - Preparan y inyectan contexto automáticamente
2. **`invoked`**: Se ejecutan DESPUÉS de que el agente responde
   - Extraen y guardan nueva información automáticamente

**Ventaja**: TODO es automático, sin código repetitivo.

**Caso Star Trek:** Data ejecuta rutinas de actualización de memoria automáticamente después de cada conversación.

In [14]:
@dataclass
class UsuarioMemoria:
    """Almacena información del usuario"""
    nombre: Optional[str] = None
    preferencias: list = None
    saludos_anteriores: int = 0
    
    def __post_init__(self):
        if self.preferencias is None:
            self.preferencias = []

class MemoriaUsuarioContextProvider:
    """
    Context Provider que aprende automáticamente.
    
    FASE 1 - invoking(): ANTES de invocar agente
      → Prepara contexto automáticamente
    
    FASE 2 - invoked(): DESPUÉS de que agente responde
      → Extrae información y actualiza memoria
    """
    
    def __init__(self):
        self.memoria = UsuarioMemoria()
        print("🧠 Context Provider de Usuario inicializado")
    
    async def invoking(self, messages, context):
        """Se ejecuta ANTES de invocar el agente"""
        print("\n🔄 [INVOKING] Preparando contexto automáticamente...")
        
        contexto_str = self._generar_contexto()
        
        if contexto_str:
            print(f"   ✓ Contexto disponible: {contexto_str[:50]}...")
            return contexto_str
        else:
            print("   ℹ️ Sin memoria anterior (primera interacción)")
            return ""
    
    async def invoked(self, request_messages, response_text):
        """Se ejecuta DESPUÉS de que el agente responde"""
        print("\n🧠 [INVOKED] Aprendiendo del resultado...")
        self._extraer_informacion(request_messages, response_text)
        print(f"   ✓ Memoria actualizada: {self.memoria}")
    
    def _generar_contexto(self) -> str:
        """Genera contexto a inyectar"""
        if not self.memoria.nombre and not self.memoria.preferencias:
            return ""
        
        contexto = "\n=== CONTEXTO AUTOMÁTICO ==="
        if self.memoria.nombre:
            contexto += f"\n👤 Nombre: {self.memoria.nombre}"
        contexto += f"\n🔄 Interacciones: {self.memoria.saludos_anteriores}"
        contexto += "\n===========================\n"
        return contexto
    
    def _extraer_informacion(self, mensajes, respuesta):
        """Extrae información nueva"""
        texto_completo = str(mensajes) + " " + respuesta
        
        if "me llamo" in texto_completo.lower():
            palabras = texto_completo.split()
            for i, palabra in enumerate(palabras):
                if "llamo" in palabra.lower() and i + 1 < len(palabras):
                    nombre_potencial = palabras[i + 1].strip(".,;:")
                    if nombre_potencial.isalpha() and len(nombre_potencial) > 2:
                        self.memoria.nombre = nombre_potencial
                        print(f"   ✓ Aprendido nombre: {nombre_potencial}")
        
        self.memoria.saludos_anteriores += 1

print("✅ Clase MemoriaUsuarioContextProvider definida")

✅ Clase MemoriaUsuarioContextProvider definida


In [16]:
print("\n" + "="*70)
print("🎬 ESCENA: Data Activando Context Provider")
print("Situación: Sistema aprende automáticamente sin código extra")
print("="*70)

context_provider = MemoriaUsuarioContextProvider()
thread_cp = agent.get_new_thread()

print("\n\n1️⃣ PRIMERA INTERACCIÓN - El usuario se presenta")
print("-" * 70)

print("\n📨 USUARIO: Me llamo Marina. ¿Cuál es mi nombre?")

# Fase 1: invoking (antes)
contexto_pre1 = await context_provider.invoking([], {})

# Invocar agente
mensaje1 = "Me llamo Marina. ¿Cuál es mi nombre?"
resultado1 = await agent.run(mensaje1, thread=thread_cp)
print(f"\n💬 Agente: {resultado1.text}...")

# Fase 2: invoked (después)
await context_provider.invoked([mensaje1], resultado1.text)

print("\n\n2️⃣ SEGUNDA INTERACCIÓN - Context Provider inyecta automáticamente")
print("-" * 70)

print("\n📨 USUARIO: ¿Recuerdas mi nombre?")

# Fase 1: invoking (antes) - ahora sí tiene información
contexto_pre2 = await context_provider.invoking([], {})

# Mensaje con contexto inyectado automáticamente
mensaje2 = "¿Recuerdas mi nombre?"
mensaje_con_contexto = f"{contexto_pre2}\n\nUSUARIO: {mensaje2}"
resultado2 = await agent.run(mensaje_con_contexto, thread=thread_cp)
print(f"\n💬 Agente: {resultado2.text}...")

# Fase 2: invoked (después)
await context_provider.invoked([mensaje2], resultado2.text)

print("\n\n" + "="*70)
print("✅ DEMOSTRACIÓN COMPLETADA")
print("="*70)
print("""
📊 Comparación:

❌ SIN Context Provider (manual):
   • Tienes que inyectar contexto EN CADA MENSAJE
   • Código repetitivo y propenso a errores
   • Lógica esparcida por el código

✅ CON Context Provider (automático):
   • invoking() inyecta AUTOMÁTICAMENTE
   • invoked() aprende AUTOMÁTICAMENTE
   • Código limpio y encapsulado
   • TODO sucede sin intervención manual
""")


🎬 ESCENA: Data Activando Context Provider
Situación: Sistema aprende automáticamente sin código extra
🧠 Context Provider de Usuario inicializado


1️⃣ PRIMERA INTERACCIÓN - El usuario se presenta
----------------------------------------------------------------------

📨 USUARIO: Me llamo Marina. ¿Cuál es mi nombre?

🔄 [INVOKING] Preparando contexto automáticamente...
   ℹ️ Sin memoria anterior (primera interacción)

💬 Agente: Tu nombre es Marina....

🧠 [INVOKED] Aprendiendo del resultado...
   ✓ Aprendido nombre: Marina
   ✓ Memoria actualizada: UsuarioMemoria(nombre='Marina', preferencias=[], saludos_anteriores=1)


2️⃣ SEGUNDA INTERACCIÓN - Context Provider inyecta automáticamente
----------------------------------------------------------------------

📨 USUARIO: ¿Recuerdas mi nombre?

🔄 [INVOKING] Preparando contexto automáticamente...
   ✓ Contexto disponible: 
=== CONTEXTO AUTOMÁTICO ===
👤 Nombre: Marina
🔄 In...

💬 Agente: Sí, tu nombre es Marina....

🧠 [INVOKED] Aprendiendo del re

---

# ⚖️ Comparación General de Estrategias

## 📊 Tabla Comparativa

In [17]:
comparacion = {
    "Estrategia": [
        "Resumen Inteligente con IA",
        "Pizarra de Misión",
        "Memoria a Largo Plazo",
        "Context Providers"
    ],
    "Complejidad": [
        "Alta",
        "Media",
        "Media",
        "Alta"
    ],
    "Automatización": [
        "Automática",
        "Manual",
        "Manual",
        "Muy Automática"
    ],
    "Preserva Info": [
        "✅ Sí (resumida)",
        "✅ Sí",
        "✅ Sí",
        "✅ Sí"
    ],
    "Caso de Uso": [
        "Conversaciones muy largas",
        "Info crítica compartida",
        "Personalización persistente",
        "Producción enterprise"
    ]
}

df = pd.DataFrame(comparacion)
print("\n📊 COMPARACIÓN DE ESTRATEGIAS DE GESTIÓN DE MEMORIA\n")
print(df.to_string(index=False))


📊 COMPARACIÓN DE ESTRATEGIAS DE GESTIÓN DE MEMORIA

                Estrategia Complejidad Automatización   Preserva Info                 Caso de Uso
Resumen Inteligente con IA        Alta     Automática ✅ Sí (resumida)   Conversaciones muy largas
         Pizarra de Misión       Media         Manual            ✅ Sí     Info crítica compartida
     Memoria a Largo Plazo       Media         Manual            ✅ Sí Personalización persistente
         Context Providers        Alta Muy Automática            ✅ Sí       Producción enterprise


---

## 🖖 Sabiduría Final de Spock

> *"La gestión de memoria no es una ciencia exacta, sino un arte de equilibrio:*
> - **Simplicidad** cuando sea suficiente
> - **Inteligencia** cuando sea necesario  
> - **Persistencia** cuando sea crítico
> - **Automatización** cuando mejore la eficiencia*
>
> *La estrategia óptima depende del contexto de tu misión específica."*

---

## ✅ Tarea 7 Completada

### Has explorado:
- ✅ Reducción automática con ejemplos prácticos de agente
- ✅ Resumen inteligente con IA reduciendo tokens
- ✅ Serialización de threads entre turnos
- ✅ Pizarra de misión para información crítica
- ✅ Memoria a largo plazo con personalización
- ✅ Context Providers para automatización total
- ✅ Comparación y recomendaciones para cada caso

### Próximos pasos:
1. **Experimenta** combinando estrategias según tu caso
2. **Mide** el uso de tokens y ajusta umbrales
3. **Personaliza** según tus usuarios específicos
4. **Implementa** Context Providers para producción

🌌 **Misión completada - La Flota Estelar está orgullosa de tu progreso**

🖖 **Larga vida y prosperidad**